#  📌 협업 필터링 애니메이션 추천(SVD)

### ✏️ DB 연결

In [26]:
import dotenv
import os
from pymongo import MongoClient

# 환경변수 불러오기
dotenv.load_dotenv(dotenv.find_dotenv())
USER = os.environ["MONGODB_USER"] # MongoDB user
PASSWORD = os.environ["MONGODB_PW"] # MongoDB password
PORT = int(os.environ["MONGODB_PORT"]) # MongoDB port

# DB 연결
client = MongoClient("mongodb://" + USER + ":" + PASSWORD + "@j7e104.p.ssafy.io", PORT)

db = client.animation
dbcol_detail = db.ani_info
dbcol_review = db.review

### ✏️ import

In [27]:
import pandas as pd
import numpy as np
from scipy.sparse.linalg import svds

### ✏️ DataFrame 가져오고 columns 수정

In [28]:
# 애니 정보 불러오기
ani_df = pd.DataFrame(dbcol_detail.find({},{"id":1, "name":1, "series_id":1}))
ani_df.drop('_id', axis = 1, inplace = True)
ani_df.columns = ["ani_id", "ani_name", "series_id"]
# ani_df

In [29]:
# 평가 데이터 불러오기
profile = list(dbcol_review.find({}))
rating_df = pd.DataFrame(profile)
rating_df = rating_df[["profile", "animation", "score"]]
rating_df.columns = ["user_id", "ani_id", "score"]
# rating_df = rating_df.sort_values('user_id')
# rating_df

### ✏️ Matrix로 변경

In [30]:
# 사용자 - 애니 pivot table 생성
user_ani_ratings_df = rating_df.pivot(
    index='user_id',
    columns='ani_id',
    values='score'
).fillna(0)
# user_ani_ratings_df

In [31]:
# matrix는 pivot_table 값을 numpy matrix로 만든 것 
matrix = user_ani_ratings_df.values

# user_ratings_mean은 사용자의 평균 평점 
user_ratings_mean = np.mean(matrix, axis = 1)

# R_user_mean : 사용자-애니에 대해 사용자 평균 평점을 뺀 것.
matrix_user_mean = matrix - user_ratings_mean.reshape(-1, 1)

# matrix
# matrix.shape #(91423, 4586)
# user_ratings_mean.shape #(91423,)
# matrix_user_mean.shape #(91423, 4586)
# pd.DataFrame(matrix_user_mean, index = user_ani_ratings_df.index, columns = user_ani_ratings_df.columns).head()

여기까지 진행하며 사용자가 애니에 대해 남긴 평점을 변경

### ✏️ SVD를 이용해 Matrix Factorization
- spicy를 이용해서 Truncated SVD를 구하기
- scikit learn에서 제공하는 Truncated SVD는 U, Sigma, Vt 반환 값을 제공x
- spicy에서 제공하는 TruncatedSVD는 scipy.sparse.linalg.svds를 이용하면 제공o

In [32]:
# scipy에서 제공해주는 svd.  
# U 행렬, sigma 행렬, V 전치 행렬을 반환.

U, sigma, Vt = svds(matrix_user_mean, k = 12)
# print(U.shape) #(91423, 12)
# print(sigma.shape) #(12,)
# print(Vt.shape) #(12, 4586)

In [33]:
# 위는 0이 아닌 값만 포함되었기에 0이 포함된 대칭행렬로 diag를 이용해 변환
sigma = np.diag(sigma)
# sigma.shape #(12, 12)
# sigma[0]
# sigma[1]

In [34]:
# U, Sigma, Vt의 내적을 수행하면, 다시 원본 행렬로 복원이 된다. 
# 거기에 아까 평균을 빼었으니 다시 사용자 평균 rating 더한다. 
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, index = user_ani_ratings_df.index, columns = user_ani_ratings_df.columns)
# df_svd_preds.head()
# df_svd_preds.shape #(91423, 4586)
# df_svd_preds.loc[15].sort_values(ascending=False)

### ✏️ SVD를 활용한 애니메이션 추천
- 사용자 아이디에 SVD로 나온 결과의 애니 평점이 가장 높은 데이터 순으로 정렬
- 사용자가 본 애니를 제외
- 사용자가 안 본 애니에서 평점이 높은 것을 반환
- 같은 시리즈인 것이 이미 추천되었다면 추가 안 함
- 30개의 애니메이션 아이디를 반환

In [35]:
id_to_idx = dict(zip(ani_df["ani_id"], ani_df.index))
idx_to_series = dict(zip(ani_df.index, ani_df["series_id"]))
id_to_series = dict(zip(ani_df["ani_id"], ani_df["series_id"]))

In [36]:
def recommend_ani(df_svd_preds, userId, ori_ani_df, ori_score_df):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 애니 데이터 정렬 -> 애니 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[userId].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_score_df[ori_score_df.user_id == userId]
    
    # 위에서 뽑은 user_data와 원본 애니 데이터를 합친다. 
    user_history = user_data.merge(ori_ani_df, on = 'ani_id').sort_values(['score'], ascending=False)
    
    # 유저가 이미 본 애니 id를 뽑는다. (TF-IDF에 사용)
    user_history_list = user_history['ani_id'].values.tolist()
    
    # 유저가 이미 본 애니 series id를 뽑는다.
    user_history_series_list_all = user_history['series_id'].values.tolist()
    user_history_series_list = []
    for id in user_history_series_list_all:
        # 시리즈가 미리 추가되어 있지 않고 nan이 아니면 히스토리 시리즈 리스트에 추가
        if id not in user_history_series_list and np.isnan(id) == False:
            user_history_series_list.append(id)
    
    # 원본 애니 데이터에서 사용자가 본 애니 데이터를 제외한 데이터를 추출
    recommendations = ori_ani_df[~ori_ani_df['ani_id'].isin(user_history['ani_id'])]
    
    # 사용자의 애니 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on = 'ani_id')
    
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendation_list_all = list(recommendations.sort_values(userId, ascending = False)['ani_id'])

    # 추천 애니메이션 저장 리스트
    recommendation_list = []
    
    # 시리즈 저장 리스트
    series_list = user_history_series_list[:]
    
    for id in recommendation_list_all:
        
        # 선택한 애니메이션의 인덱스 가져옴
        idx = id_to_idx[id]

        # 해당 인덱스의 시리즈 아이디 가져옴
        series = idx_to_series[idx]
        
        # 시리즈가 미리 추가되어 있지 않다면 추천결과에 추가
        if series not in series_list:
            recommendation_list.append(id)
            # nan이 아니면 시리즈 리스트에 추가
            if np.isnan(series) == False:
                series_list.append(series)
        
        # 30개가 추천되면 종료
        if len(recommendation_list) == 30:
            return user_history_list, recommendation_list

In [37]:
input_user_id = 4523846

In [38]:
user_history_list, recommendation_list = recommend_ani(df_svd_preds, input_user_id, ani_df, rating_df)

In [39]:
user_history_list

[39986,
 39431,
 25049,
 40684,
 39857,
 40249,
 38258,
 23698,
 40532,
 39996,
 40575,
 40825,
 39730,
 40372,
 36775,
 39072,
 40655,
 40192,
 40562,
 39935]

In [40]:
recommendation_list

[40630,
 40510,
 39070,
 23661,
 39081,
 40794,
 40153,
 39992,
 40163,
 39985,
 38921,
 40159,
 40790,
 40829,
 39856,
 39745,
 40276,
 40394,
 40272,
 39648,
 40682,
 40266,
 39847,
 40533,
 40261,
 40692,
 40693,
 39731,
 40181,
 28765]

### ✏️ 참고

##### 이미 본 애니들과 추천 애니들을 보기 위한 코드
```python
def recommend_ani(df_svd_preds, userId, ori_ani_df, ori_score_df, num_recommendations):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 애니 데이터 정렬 -> 애니 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[userId].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_score_df[ori_score_df.user_id == userId]
    
    # 위에서 뽑은 user_data와 원본 애니 데이터를 합친다. 
    user_history = user_data.merge(ori_ani_df, on = 'ani_id').sort_values(['score'], ascending=False)
    
    # 원본 애니 데이터에서 사용자가 본 애니 데이터를 제외한 데이터를 추출
    recommendations = ori_ani_df[~ori_ani_df['ani_id'].isin(user_history['ani_id'])]
    
    # 사용자의 애니 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'ani_id')
    
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {userId: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]  

    return return user_history, recommendations

already_rated, predictions = recommend_ani(df_svd_preds, input_user_id, ani_df, rating_df, 30)
print(already_rated)
print(predictions)
```
#####  num_recommendations개를 추천받는 코드
```python
def recommend_ani(df_svd_preds, userId, ori_ani_df, ori_score_df, num_recommendations):
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 애니 데이터 정렬 -> 애니 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[userId].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_score_df[ori_score_df.user_id == userId]
    
    # 위에서 뽑은 user_data와 원본 애니 데이터를 합친다. 
    user_history = user_data.merge(ori_ani_df, on = 'ani_id').sort_values(['score'], ascending=False)
    
    # 원본 애니 데이터에서 사용자가 본 애니 데이터를 제외한 데이터를 추출
    recommendations = ori_ani_df[~ori_ani_df['ani_id'].isin(user_history['ani_id'])]
    
    # 사용자의 애니 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'ani_id')
    
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendation_list = list(recommendations.sort_values(userId, ascending = False)['ani_id'])[:num_recommendations]

    return recommendation_list
```
##### 시리즈 체크 코드
```python
id_to_idx = dict(zip(ani_df["ani_id"], ani_df.index))
idx_to_series = dict(zip(ani_df.index, ani_df["series_id"]))
id_to_series = dict(zip(ani_df["ani_id"], ani_df["series_id"]))

def series_check(recommendation_list):
    # 추천 애니메이션 저장 리스트
    recommendation = []
    # 시리즈 저장 리스트
    series_list = []
    
    for id in recommendation_list:
        # 선택한 애니메이션의 인덱스 가져옴
        idx = id_to_idx[id]

        # 해당 인덱스의 시리즈 아이디 가져옴
        series = idx_to_series[idx]
        
        if series not in series_list:
            recommendation.append(id)
            if np.isnan(series) == False:
                series_list.append(series)

    return recommendation
```

### ✏️ SVD를 활용한 30개의 추천에서 TF-IDF를 활용한 14개 추천

In [41]:
dbcol_feat = db.ani_feature

In [42]:
feat_df = pd.DataFrame(dbcol_feat.find({},{"id":1, "feat_str":1}))

In [43]:
id_to_idx = dict(zip(feat_df["id"], feat_df.index))
idx_to_feat = dict(zip(feat_df.index, feat_df["feat_str"]))

In [44]:
tf_idf_list = list()
user_feat = ''
for i in user_history_list:
    idx = id_to_idx[i]
    feat = idx_to_feat[idx]
    user_feat += feat
    user_feat += ' '
tf_idf_list.append(user_feat)
# print(tf_idf_list)

In [45]:
for j in recommendation_list:
    idx = id_to_idx[j]
    reco_feat = idx_to_feat[idx]
    tf_idf_list.append(reco_feat)
# print(tf_idf_list)

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# TF-IDF 분석
tf_idf = TfidfVectorizer()
tf_idf_matrix = tf_idf.fit_transform(tf_idf_list)
print("TF-IDF 행렬의 크기(shape): ", tf_idf_matrix.shape)

cosine_sim = cosine_similarity(tf_idf_matrix, tf_idf_matrix)
print("코사인 유사도 연산 결과: ",cosine_sim.shape)

TF-IDF 행렬의 크기(shape):  (31, 1344)
코사인 유사도 연산 결과:  (31, 31)


In [47]:
# cosine_sim
cosine_sim[0]

array([1.        , 0.06303984, 0.11713234, 0.06221434, 0.03477249,
       0.06642304, 0.16320931, 0.09846057, 0.09324396, 0.14613305,
       0.07312431, 0.09807444, 0.14761016, 0.21761293, 0.05648138,
       0.13864421, 0.05311521, 0.12725926, 0.10011708, 0.1147141 ,
       0.08505919, 0.14569474, 0.01964442, 0.03615426, 0.04768092,
       0.07229158, 0.02547821, 0.09490013, 0.14851751, 0.14792421,
       0.10119623])

In [48]:
# 유저가 시청한 모든 자료의 합인 첫 행과의 유사도를 가져온다.
sim_scores = list(enumerate(cosine_sim[0]))
# 유사도에 따라 애니 정렬 후 14개 추출
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
# print(sim_scores)
sim_scores = sim_scores[1:15]
# print(sim_scores)
# 추천 애니메이션 저장 리스트
id_list = list()
# 14개 추출
for num in range(14):
#     print(sim_scores[num][0])
#     print(recommendation_list[sim_scores[num][0]-1])
    id_list.append(recommendation_list[sim_scores[num][0]-1])
print(id_list)

[40790, 40794, 39731, 40181, 40159, 40163, 40682, 39856, 40276, 40510, 40272, 28765, 40394, 40153]


In [49]:
# for result in id_list:
id_to_name = dict(zip(ani_df["ani_id"], ani_df["ani_name"]))
for result in id_list:
    print(id_to_name[result])

아하렌 양은 알 수가 없어
귀엽기만 한 게 아닌 시키모리 양
역시 내 청춘 러브코메디는 잘못됐다. 완
괴물사변
5등분의 신부 ∬
전생했더니 슬라임이었던 건에 대하여 2기 1부
아케비의 세일러복
여친, 빌리겠습니다
(자막) 나의 히어로 아카데미아 5기
무직전생 : 이세계에 갔으면 최선을 다한다 2기
불멸의 그대에게 part 1
암살교실 2기
도쿄 리벤저스 part 2
Re : 제로부터 시작하는 이세계 생활 2기 part 2


In [50]:
for result in user_history_list:
    print(id_to_name[result])

주술회전 part 1
(무삭제) 귀멸의 칼날
니세코이: (2기)
장난을 잘 치는 타카기 양 3기
우자키 양은 놀고 싶어!
괴롭히지 말아요, 나가토로 양
장난을 잘 치는 타카기 양 1기
니세코이
선배가 짜증나는 후배이야기
어쨌든 귀여워
귀멸의 칼날 : 무한열차편
카구야 님은 고백받고 싶어 -울트라 로맨틱-
카구야 님은 고백받고 싶어 ~천재들의 연애 두뇌전~ 2기
코바야시네 메이드래곤 S
코바야시네 메이드래곤
카구야 님은 고백받고 싶어 ~천재들의 연애 두뇌전~ 1기
그 비스크 돌은 사랑을 한다
주술회전 part 2
귀멸의 칼날 : 환락의 거리편
바케모노가타리 - 판권 부활
